# Training of CNN with 1x1 filter

Zašto korištenje 1x1 filtera?
Zato da algoritam pronađe vezu između true ishoda i dubine ulaza.

Planiram koristiti:
- INICIJALIZACIJA??
- aktivacijska funkcija: Relu
- normalizacija: Batch Normalization - zapravo, s obzirom da je Batch norm. koristan prvenstveno kod dubokih mreža, pokušati ću prvo bez toga, da vidim kako funkcionira.
- regularizacija: Dropout
- OPTIMIZER? ADAM?

S obzirom da ima malo podataka, razmisli može li se nekako povećati broj podataka data augmentation tehnikama.

In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
import csv

from sklearn.metrics import accuracy_score

# Common imports
import numpy as np
import os

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


%matplotlib inline

In [2]:
print ('DATA PREPARATION')
vege_csv = csv.reader(open('vegetacija.csv', newline=''), delimiter=' ', quotechar='|')

vege_array = []


for row in vege_csv:
	red_array = []
	items = row[0].split(';')
	for i in items:
		red_array.append(i)

	vege_array.append(red_array)


klase = []
values = []

vege_array_fix = vege_array[:-4]

print ('length of the fixed array is ', len (vege_array_fix))

for r in vege_array_fix:
    value_row = []
    i = 0
    while i < len(r):
        if i == 0:
            if r[i] == 'X':
                klase.append(0)
            if r[i] == 'V':
                klase.append(1)
            if r[i] == 'N':
                klase.append(2)
        else:
            value_row.append(float(r[i])/1000)
        i = i + 1
    values.append(value_row)

print ('test : ', values[10000])

DATA PREPARATION
length of the fixed array is  199400
test :  [0.339, 0.201, 0.238, 0.278, 0.186, 0.298, 0.298, 0.253]


In [3]:
X_train, X_test, y_train, y_test = train_test_split(values, klase, test_size=0.20)

In [4]:
reset_graph()

In [5]:
height = 1
width = 1
channels = 8
# n_inputs = height * width

conv1_fmaps = 16
conv1_ksize = 1
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 1
conv2_stride = 1
conv2_pad = "SAME"

conv3_fmaps = 64
conv3_ksize = 1
conv3_stride = 1
conv3_pad = "SAME"

conv4_fmaps = 128
conv4_ksize = 1
conv4_stride = 1
conv4_pad = "SAME"

pool3_fmaps = conv4_fmaps

n_fc1 = 128
n_outputs = 3

# n_inputs = len(X_train)

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.elu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.elu, name="conv2")
conv3 = tf.layers.conv2d(conv2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad,
                         activation=tf.nn.elu, name="conv3") 
conv4 = tf.layers.conv2d(conv3, filters=conv4_fmaps, kernel_size=conv4_ksize,
                         strides=conv4_stride, padding=conv4_pad,
                         activation=tf.nn.elu, name="conv4") 


with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv4, ksize=[1, 1, 1, 1], strides=[1, 1, 1, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 1 * 1])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.elu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")
    
with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [6]:
print (len(X_train))

159520


In [7]:
X_valid = X_train[:63808]
X_train = X_train[63808:]

y_valid = y_train[:63808]
y_train = y_train[63808:]

In [8]:
train_array_length = len(X_train)
print (train_array_length)

95712


In [9]:
#moja egzekucijska funkcija

# n_epochs = 100
# batch_size = 20

# with tf.Session() as sess:
#     init.run()
#     for epoch in range(n_epochs):
#         for iteration in range(int(len(X_train)/batch_size)):
#             X_batch, y_batch = X_train.train.next_batch(batch_size)
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
#         loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
#         if loss_val < best_loss:
#             save_path = saver.save(sess, "./vege_model.ckpt")
#             best_loss = loss_val
#             checks_without_progress = 0
#         else:
#             checks_without_progress += 1
#             if checks_without_progress > max_checks_without_progress:
#                 print("Early stopping!")
#                 break
#         print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
#             epoch, loss_val, best_loss, acc_val * 100))

# with tf.Session() as sess:
#     saver.restore(sess, "./vege_model.ckpt")
#     acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
#     print("Final test accuracy: {:.2f}%".format(acc_test * 100))

In [10]:

# n_epochs = 10
# batch_size = 20

# max_checks_without_progress = 20
# checks_without_progress = 0
# best_loss = np.infty

# with tf.Session() as sess:
#     init.run()

#     for epoch in range(n_epochs):
#         rnd_idx = np.random.permutation(train_array_length)
#         for rnd_indices in np.array_split(rnd_idx, train_array_length // batch_size):
#             print (rnd_indices)

In [11]:

n_epochs = 1000
batch_size = 50

max_checks_without_progress = 300
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch = []
            y_batch = []
            for i in rnd_indices:
                X_batch.append(X_train[i]) 
                y_batch.append(y_train[i])
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid, y: y_valid})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./vege_model.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./vege_model.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.591130	Best loss: 0.591130	Accuracy: 81.75%
1	Validation loss: 0.583252	Best loss: 0.583252	Accuracy: 81.74%
2	Validation loss: 0.581086	Best loss: 0.581086	Accuracy: 81.74%
3	Validation loss: 0.572437	Best loss: 0.572437	Accuracy: 81.74%
4	Validation loss: 0.562935	Best loss: 0.562935	Accuracy: 81.75%
5	Validation loss: 0.557396	Best loss: 0.557396	Accuracy: 81.75%
6	Validation loss: 0.577650	Best loss: 0.557396	Accuracy: 81.77%
7	Validation loss: 0.555130	Best loss: 0.555130	Accuracy: 81.74%
8	Validation loss: 0.549425	Best loss: 0.549425	Accuracy: 81.77%
9	Validation loss: 0.545197	Best loss: 0.545197	Accuracy: 81.81%
10	Validation loss: 0.560113	Best loss: 0.545197	Accuracy: 81.73%
11	Validation loss: 0.542664	Best loss: 0.542664	Accuracy: 81.79%
12	Validation loss: 0.538474	Best loss: 0.538474	Accuracy: 81.78%
13	Validation loss: 0.549106	Best loss: 0.538474	Accuracy: 81.80%
14	Validation loss: 0.553222	Best loss: 0.538474	Accuracy: 81.74%
15	Validation loss: 

124	Validation loss: 0.439642	Best loss: 0.434247	Accuracy: 83.45%
125	Validation loss: 0.450818	Best loss: 0.434247	Accuracy: 83.17%
126	Validation loss: 0.484829	Best loss: 0.434247	Accuracy: 82.28%
127	Validation loss: 0.456386	Best loss: 0.434247	Accuracy: 82.96%
128	Validation loss: 0.439145	Best loss: 0.434247	Accuracy: 83.35%
129	Validation loss: 0.474543	Best loss: 0.434247	Accuracy: 82.34%
130	Validation loss: 0.449868	Best loss: 0.434247	Accuracy: 82.86%
131	Validation loss: 0.439556	Best loss: 0.434247	Accuracy: 83.12%
132	Validation loss: 0.451133	Best loss: 0.434247	Accuracy: 83.00%
133	Validation loss: 0.451305	Best loss: 0.434247	Accuracy: 83.09%
134	Validation loss: 0.442865	Best loss: 0.434247	Accuracy: 83.20%
135	Validation loss: 0.443583	Best loss: 0.434247	Accuracy: 82.92%
136	Validation loss: 0.439021	Best loss: 0.434247	Accuracy: 83.25%
137	Validation loss: 0.446330	Best loss: 0.434247	Accuracy: 83.00%
138	Validation loss: 0.488038	Best loss: 0.434247	Accuracy: 82

247	Validation loss: 0.416223	Best loss: 0.386979	Accuracy: 83.58%
248	Validation loss: 0.390899	Best loss: 0.386979	Accuracy: 84.66%
249	Validation loss: 0.385019	Best loss: 0.385019	Accuracy: 84.45%
250	Validation loss: 0.404515	Best loss: 0.385019	Accuracy: 84.29%
251	Validation loss: 0.411195	Best loss: 0.385019	Accuracy: 84.03%
252	Validation loss: 0.394200	Best loss: 0.385019	Accuracy: 84.42%
253	Validation loss: 0.441013	Best loss: 0.385019	Accuracy: 83.85%
254	Validation loss: 0.444379	Best loss: 0.385019	Accuracy: 83.48%
255	Validation loss: 0.410981	Best loss: 0.385019	Accuracy: 83.90%
256	Validation loss: 0.381196	Best loss: 0.381196	Accuracy: 84.90%
257	Validation loss: 0.427444	Best loss: 0.381196	Accuracy: 83.64%
258	Validation loss: 0.404270	Best loss: 0.381196	Accuracy: 84.27%
259	Validation loss: 0.391043	Best loss: 0.381196	Accuracy: 84.15%
260	Validation loss: 0.391520	Best loss: 0.381196	Accuracy: 84.46%
261	Validation loss: 0.399551	Best loss: 0.381196	Accuracy: 84

370	Validation loss: 0.350565	Best loss: 0.339601	Accuracy: 86.07%
371	Validation loss: 0.423764	Best loss: 0.339601	Accuracy: 83.59%
372	Validation loss: 0.364246	Best loss: 0.339601	Accuracy: 85.44%
373	Validation loss: 0.396449	Best loss: 0.339601	Accuracy: 84.74%
374	Validation loss: 0.356085	Best loss: 0.339601	Accuracy: 85.81%
375	Validation loss: 0.364939	Best loss: 0.339601	Accuracy: 85.61%
376	Validation loss: 0.337183	Best loss: 0.337183	Accuracy: 86.64%
377	Validation loss: 0.430862	Best loss: 0.337183	Accuracy: 83.95%
378	Validation loss: 0.359297	Best loss: 0.337183	Accuracy: 85.72%
379	Validation loss: 0.353642	Best loss: 0.337183	Accuracy: 86.16%
380	Validation loss: 0.364382	Best loss: 0.337183	Accuracy: 85.83%
381	Validation loss: 0.360661	Best loss: 0.337183	Accuracy: 85.80%
382	Validation loss: 0.343811	Best loss: 0.337183	Accuracy: 86.42%
383	Validation loss: 0.356685	Best loss: 0.337183	Accuracy: 85.96%
384	Validation loss: 0.352990	Best loss: 0.337183	Accuracy: 85

493	Validation loss: 0.337255	Best loss: 0.316595	Accuracy: 86.98%
494	Validation loss: 0.349445	Best loss: 0.316595	Accuracy: 86.60%
495	Validation loss: 0.310824	Best loss: 0.310824	Accuracy: 87.88%
496	Validation loss: 0.333605	Best loss: 0.310824	Accuracy: 87.31%
497	Validation loss: 0.329040	Best loss: 0.310824	Accuracy: 87.10%
498	Validation loss: 0.364653	Best loss: 0.310824	Accuracy: 85.88%
499	Validation loss: 0.323567	Best loss: 0.310824	Accuracy: 87.51%
500	Validation loss: 0.322432	Best loss: 0.310824	Accuracy: 87.31%
501	Validation loss: 0.315496	Best loss: 0.310824	Accuracy: 87.75%
502	Validation loss: 0.324038	Best loss: 0.310824	Accuracy: 87.14%
503	Validation loss: 0.318599	Best loss: 0.310824	Accuracy: 87.84%
504	Validation loss: 0.346529	Best loss: 0.310824	Accuracy: 86.66%
505	Validation loss: 0.321010	Best loss: 0.310824	Accuracy: 87.41%
506	Validation loss: 0.331948	Best loss: 0.310824	Accuracy: 86.82%
507	Validation loss: 0.386770	Best loss: 0.310824	Accuracy: 85

616	Validation loss: 0.358636	Best loss: 0.307262	Accuracy: 86.85%
617	Validation loss: 0.309078	Best loss: 0.307262	Accuracy: 88.45%
618	Validation loss: 0.332799	Best loss: 0.307262	Accuracy: 87.61%
619	Validation loss: 0.343771	Best loss: 0.307262	Accuracy: 87.20%
620	Validation loss: 0.331045	Best loss: 0.307262	Accuracy: 87.67%
621	Validation loss: 0.351618	Best loss: 0.307262	Accuracy: 87.15%
622	Validation loss: 0.389416	Best loss: 0.307262	Accuracy: 86.19%
623	Validation loss: 0.391259	Best loss: 0.307262	Accuracy: 86.42%
624	Validation loss: 0.392152	Best loss: 0.307262	Accuracy: 85.93%
625	Validation loss: 0.378176	Best loss: 0.307262	Accuracy: 86.13%
626	Validation loss: 0.347190	Best loss: 0.307262	Accuracy: 87.28%
627	Validation loss: 0.333633	Best loss: 0.307262	Accuracy: 87.84%
628	Validation loss: 0.310182	Best loss: 0.307262	Accuracy: 88.32%
629	Validation loss: 0.335600	Best loss: 0.307262	Accuracy: 87.64%
630	Validation loss: 0.332772	Best loss: 0.307262	Accuracy: 88

739	Validation loss: 0.317692	Best loss: 0.305202	Accuracy: 88.63%
740	Validation loss: 0.484173	Best loss: 0.305202	Accuracy: 84.48%
741	Validation loss: 0.347807	Best loss: 0.305202	Accuracy: 87.82%
742	Validation loss: 0.327259	Best loss: 0.305202	Accuracy: 88.17%
743	Validation loss: 0.332036	Best loss: 0.305202	Accuracy: 87.98%
744	Validation loss: 0.396434	Best loss: 0.305202	Accuracy: 86.48%
745	Validation loss: 0.335553	Best loss: 0.305202	Accuracy: 87.77%
746	Validation loss: 0.319682	Best loss: 0.305202	Accuracy: 88.75%
747	Validation loss: 0.322922	Best loss: 0.305202	Accuracy: 88.21%
748	Validation loss: 0.347642	Best loss: 0.305202	Accuracy: 87.71%
749	Validation loss: 0.358795	Best loss: 0.305202	Accuracy: 87.63%
750	Validation loss: 0.341849	Best loss: 0.305202	Accuracy: 88.35%
751	Validation loss: 0.324176	Best loss: 0.305202	Accuracy: 88.38%
752	Validation loss: 0.381119	Best loss: 0.305202	Accuracy: 87.09%
753	Validation loss: 0.356206	Best loss: 0.305202	Accuracy: 87

862	Validation loss: 0.339306	Best loss: 0.301913	Accuracy: 88.37%
863	Validation loss: 0.363276	Best loss: 0.301913	Accuracy: 87.74%
864	Validation loss: 0.354710	Best loss: 0.301913	Accuracy: 87.72%
865	Validation loss: 0.364854	Best loss: 0.301913	Accuracy: 88.04%
866	Validation loss: 0.346760	Best loss: 0.301913	Accuracy: 88.18%
867	Validation loss: 0.365186	Best loss: 0.301913	Accuracy: 88.05%
868	Validation loss: 0.371527	Best loss: 0.301913	Accuracy: 87.49%
869	Validation loss: 0.390228	Best loss: 0.301913	Accuracy: 87.00%
870	Validation loss: 0.325003	Best loss: 0.301913	Accuracy: 88.55%
871	Validation loss: 0.405828	Best loss: 0.301913	Accuracy: 87.04%
872	Validation loss: 0.455212	Best loss: 0.301913	Accuracy: 85.32%
873	Validation loss: 0.327001	Best loss: 0.301913	Accuracy: 88.66%
874	Validation loss: 0.337975	Best loss: 0.301913	Accuracy: 88.57%
875	Validation loss: 0.358314	Best loss: 0.301913	Accuracy: 88.36%
876	Validation loss: 0.335767	Best loss: 0.301913	Accuracy: 88

985	Validation loss: 0.354951	Best loss: 0.301913	Accuracy: 88.03%
986	Validation loss: 0.401924	Best loss: 0.301913	Accuracy: 86.62%
987	Validation loss: 0.395800	Best loss: 0.301913	Accuracy: 87.66%
988	Validation loss: 0.320681	Best loss: 0.301913	Accuracy: 89.38%
989	Validation loss: 0.323300	Best loss: 0.301913	Accuracy: 89.09%
990	Validation loss: 0.330352	Best loss: 0.301913	Accuracy: 89.03%
991	Validation loss: 0.346106	Best loss: 0.301913	Accuracy: 88.53%
992	Validation loss: 0.339908	Best loss: 0.301913	Accuracy: 88.70%
993	Validation loss: 0.397166	Best loss: 0.301913	Accuracy: 86.93%
994	Validation loss: 0.356075	Best loss: 0.301913	Accuracy: 88.21%
995	Validation loss: 0.333311	Best loss: 0.301913	Accuracy: 88.84%
996	Validation loss: 0.371867	Best loss: 0.301913	Accuracy: 87.83%
997	Validation loss: 0.338735	Best loss: 0.301913	Accuracy: 88.57%
998	Validation loss: 0.380457	Best loss: 0.301913	Accuracy: 87.95%
999	Validation loss: 0.360726	Best loss: 0.301913	Accuracy: 87